# Milestone Project 2

### Problem Statement
The number of RCT papers released is continuously increasing. It is difficult for researchers to keep up with the latest research in their field. This project aims to create a tool that can help researchers to keep up with the latest research in their field. The tool will be able to search for the latest RCT papers in a specific field and summarize the key findings of the papers.

### Solution
Create an NLP model to classify abstract sentences into the role they play (e.g. objective, methods, results, etc)  to enable researchers to skim through the literature (hence SkimLit 🤓🔥) and dive deeper when necessary.

> 📖 **Resources:** Before going through the code in this notebook, you might want to get a background of what we're going to be doing. To do so, spend an hour (or two) going through the following papers and then return to this notebook:
1. Where our data is coming from: [*PubMed 200k RCT: a Dataset for Sequential Sentence Classification in Medical Abstracts*](https://arxiv.org/abs/1710.06071)
2. Where our model is coming from: [*Neural networks for joint sentence
classification in medical paper abstracts*](https://arxiv.org/pdf/1612.05251.pdf).

## Topics to be covered
* Downloading a dataset ([PubMed RCT200k from GitHub](https://github.com/Franck-Dernoncourt/pubmed-rct))
* Preprocessing data for NLP models
* Setting up a series of NLP models
    * Making a baseline model (TF-IDF Multinomial Naive Bayes)
    * Deep models with different combinations of layers (Conv1D, LSTM, GRU)
* Building first multimodal model (feature extraction and token embeddings)
    * Replicating the model architecture from https://arxiv.org/abs/1612.05251
* Find the most wrong predictions
* Making predictions on PubMed abstracts
